In [1]:
import ollama
ollama.pull("llama3:8b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [2]:
# --- Classification Function ---
def classify_urdu_question(urdu_question):
    prompt = f"""You are given a question in Urdu. Your task is to classify it as either SINGLEHOP or MULTIHOP.

Definitions:
- A question is **SINGLEHOP** if it can be answered using a single fact, sentence, or document. The question may be long, but if it doesn't require combining or reasoning over multiple distinct pieces of information, it's SINGLEHOP.
- A question is **MULTIHOP** if answering it requires combining multiple facts, reasoning over several steps, or connecting pieces of information from different sources.

⚠️ Important:
- A question's length does not determine its type. A long, descriptive question can still be SINGLEHOP.
- MULTIHOP questions typically require you to first find one piece of information, then use that to find the next.

Examples:

🔹 Example 1 (SINGLEHOP - short and factual):
سوال: "پاکستان کے وزیر اعظم کا نام کیا ہے؟"
جواب: SINGLEHOP

🔹 Example 2 (SINGLEHOP - long, descriptive):
سوال: "میں بلیک ہیٹ پہنتا ہوں: بدمعاشوں کے ساتھ جدوجہد (حقیقی اور خیالی) ایک کتاب ہے جو چاک کلوسٹر مین نے لکھی ہے ، جو پہلی بار اسکربر نے شائع کی تھی۔"
جواب: SINGLEHOP

🔹 Example 3 (SINGLEHOP - event description):
سوال: "یہ فلم سنہ 2010 میں ریلیز ہوئی تھی اور اس میں 16 قتلوں کی کہانی بیان کی گئی ہے جو سنہ 1828 میں سکاٹ لینڈ کے شہر ایڈنبرا میں ہوئے تھے۔"
جواب: SINGLEHOP

🔹 Example 4 (SINGLEHOP - company and product):
سوال: "والگریریا فرانزی ایک اطالوی چمڑے کے بیگ اور سامان کی کمپنی تھی جس کی بنیاد 1864 میں روکو فرانزی نے رکھی تھی۔ گچی نے اس کمپنی کے لئے کام کیا جب تک کہ اس نے گچی کو قائم نہیں کیا۔"
جواب: SINGLEHOP

🔸 Example 5 (MULTIHOP - indirect question):
سوال: "جارج واشنگٹن کی پیدائش کے وقت امریکہ کا صدر کون تھا؟"
جواب: MULTIHOP

🔸 Example 6 (MULTIHOP - location relation):
سوال: "نلسن منڈیلا کو کس جیل میں رکھا گیا اور وہ جیل کس جزیرے پر تھی؟"
جواب: MULTIHOP

🔸 Example 7 (MULTIHOP - historical causality):
سوال: "انگریزوں کے خلاف 1857 کی جنگ کے بعد ہندوستان میں کونسا سیاسی نظام قائم ہوا؟"
جواب: MULTIHOP

🔸 Example 8 (MULTIHOP - lengthy, layered facts):
سوال: "ایپل کمپنی کے شریک بانی اسٹیو جابز نے کس یونیورسٹی سے تعلیم حاصل کی، اور اس یونیورسٹی کے سب سے معروف گریجویٹس میں سے ایک کون ہے؟"
جواب: MULTIHOP

🔸 Example 9 (MULTIHOP - lengthy with connections):
سوال: "شیکسپیئر کے وہ ڈرامے جو بادشاہوں پر مبنی ہیں، ان میں سے ایک کے مرکزی کردار کی موت کس جنگ میں ہوئی تھی؟"
جواب: MULTIHOP

---

Now read the following Urdu question and classify it:

سوال: "{urdu_question}"
جواب:"""
    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        # Normalize output
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"


In [3]:
def decompose_urdu_query(urdu_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
**Role**: You are an expert Urdu linguistic analyst specializing in question decomposition. Your task is to break down complex Urdu questions into their fundamental components.

**Task Instructions**:
1. Carefully analyze the given Urdu question to identify its core components
2. Extract exactly 2 sub-questions that:
   - Are necessary to answer the main question
   - Cover distinct aspects of the problem
   - Have clear logical progression (answer to q1 helps answer q2)
3. Both sub-questions must:
   - Be in proper Urdu language
   - Be grammatically correct
   - Be clear and concise
   - Use relevant domain terminology

**Output Format Requirements**:
- Use EXACTLY this format:
  q1: [پہلا ذیلی سوال]
  q2: [دوسرا ذیلی سوال]
- Each sub-question must be on a new line
- Do not include any additional commentary or explanation
- Do not number the questions (use only q1:/q2: prefixes)

**Example 1**:
Input: اگر لاہور میں فضائی آلودگی کی سطح دہلی سے زیادہ ہے اور فضائی آلودگی پھیپھڑوں کے کینسر کا سبب بن سکتی ہے، تو لاہور کے رہائشیوں کو کس قسم کے طبی چیک اپ کروانے چاہئیں؟
Output:
q1: لاہور اور دہلی میں فضائی آلودگی کی سطح کا موازنہ کیا ہے؟
q2: فضائی آلودگی پھیپھڑوں کے کینسر کا سبب کیسے بنتی ہے؟

**Example 2**:
Input: اگر کراچی میں بجلی کے نرخ 30% بڑھ گئے ہیں اور یہ صنعتوں کو متاثر کر رہا ہے، تو حکومت کو کون سی سبسڈیاں دینی چاہئیں؟
Output:
q1: کراچی میں بجلی کے نرخوں میں اضافے کی موجودہ شرح کیا ہے؟
q2: بجلی کے مہنگے ہونے سے صنعتوں پر کس قسم کے اثرات مرتب ہو رہے ہیں؟

**Current Task**:
Input: {urdu_query}
Output:
"""
    
    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048
            }
        )
        
        output = response['response'].strip()
        
        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()
        
        return result if len(result) == 2 else {}
    
    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}


In [4]:
def query_context_relevance_check(query_urdu: str, context_urdu: str) -> bool:
    prompt = f"""
You are a binary classifier.

Your task is to decide whether the following Urdu *context* is relevant to the Urdu *question*. You must answer ONLY with **True** or **False** — no explanation, no commentary, just one word: True or False.

Criteria:
- If the context helps answer the question directly or indirectly, reply: True
- If the context is unrelated, confusing, or insufficient, reply: False

IMPORTANT:
- Do NOT explain your answer.
- Do NOT include any additional comments.
- Just respond with: True or False

---

Question (Urdu): {query_urdu}

Context (Urdu): {context_urdu}

Answer (True/False):
"""

    try:
        response = ollama.chat(
            model="llama3:8b",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response['message']['content'].strip().lower()
        return answer == 'true'
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False


In [5]:
from sentence_transformers import SentenceTransformer
import pickle
import faiss
import os
import torch


def load_retriever(
    index_path: str,
    chunks_path: str
):
    # Initialize device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load SentenceTransformer model (E5-large)
    model = SentenceTransformer("intfloat/e5-large", device=device)
    
    # Configure for Urdu text
    model.max_seq_length = 512  # Set based on your earlier analysis
    model.tokenizer.do_lower_case = False  # Preserve Urdu characters
    
    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)
    
    return model, index, chunks_list, device



c:\hammad workings\Thesis\Multihop_Project\MultiHop_Query_Handling_in_RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model, index, chunks_list, device = load_retriever(
    index_path="../../vector_db/paragraphs/5884_paras/5884_paras_faiss_index.index",
    chunks_path="../../data_storage/Paragraph_chunks/5884_paragraphs/5884_chunks.pkl"
)

In [7]:
print(model.device)


cuda:0


In [8]:
def retrieve_documents(query,k=3):
   
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,  # Return numpy array for FAISS
        normalize_embeddings=True,
        show_progress_bar=False
    )
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Search FAISS index
    _, indices = index.search(query_embedding, k)  # Dummy variable _ for distances
    
    # Return only the chunks
    return [chunks_list[i] for i in indices[0]]


In [9]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def expand_multihop_context(intermediate_ctx, query, q1, q2, k):
    try:
        combined_query = q1 + intermediate_ctx + q2
        second_hop_contexts = retrieve_documents(combined_query, k)

        relevant_contexts = []

        with ThreadPoolExecutor() as inner_executor:
            futures = [
                inner_executor.submit(query_context_relevance_check, query, ctx)
                for ctx in second_hop_contexts
            ]

            for i, future in enumerate(as_completed(futures)):
                try:
                    if future.result():
                        relevant_contexts.append(second_hop_contexts[i])
                except Exception as e:
                    print("Error during relevance check:", e)

        return relevant_contexts

    except Exception as e:
        print("Error in expand_multihop_context:", e)
        return []


def get_context_of_multihop(query, type, k=3):
    # Step 1: Classification
    start_classification = time.time()
    classification = classify_urdu_question(query)
    classification_time = time.time() - start_classification

    if type == "easy":
        decomposition_time = 0.0
        start_retrieval = time.time()
        context = retrieve_documents(query, k)
        retrieval_time = time.time() - start_retrieval
        return context, classification, classification_time, decomposition_time, retrieval_time

    else:
        # Step 2: Decomposition
        start_decomposition = time.time()
        decomposition = decompose_urdu_query(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")
        decomposition_time = time.time() - start_decomposition

        # Step 3: First-hop retrieval for q1 and q2
        start_retrieval = time.time()
        context_q1 = retrieve_documents(q1, k)
        context_q2 = retrieve_documents(q2, k)

        # Combine base context
        main_context = context_q1 + context_q2

        additional_contexts = []

        # Expand q1
        with ThreadPoolExecutor() as executor_q1:
            futures_q1 = [
                executor_q1.submit(expand_multihop_context, ctx, query, q1, q2, k)
                for ctx in context_q1[:k]
            ]
            for future in as_completed(futures_q1):
                result = future.result()
                additional_contexts.extend(result)

        # Expand q2
        with ThreadPoolExecutor() as executor_q2:
            futures_q2 = [
                executor_q2.submit(expand_multihop_context, ctx, query, q1, q2, k)
                for ctx in context_q2[:k]
            ]
            for future in as_completed(futures_q2):
                result = future.result()
                additional_contexts.extend(result)

        # Merge all retrieved content
        main_context.extend(additional_contexts)
        retrieval_time = time.time() - start_retrieval

        return main_context, classification, classification_time, decomposition_time, retrieval_time


In [11]:
import ollama

def generate_using_llama3(context, query):
    prompt = f"""You are a helpful assistant. You will be given a context and a question, both in Urdu.
Your task is to answer the question using the context only. 
Your answer should be **clear, concise, and entirely in Urdu**.
Context:
{context}

Question:
{query}

Answer in Urdu:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print("Error during generation:", e)
        return "جواب پیدا کرنے میں خرابی ہوئی۔"



In [12]:
import time

def multihop_handling_modLQR(query, type, k=3):
    # Step 1: Get context and timings
    context, classification, classification_time, decomposition_time, retrieval_time = get_context_of_multihop(
        query, type, k=k
    )

    # Flatten context if it's a list of strings
    if isinstance(context, list):
        combined_context = "\n".join(context)
    else:
        combined_context = context

    # Step 2: Generate answer and measure time
    start_gen = time.time()
    final_answer = generate_using_llama3(combined_context,query)
    generation_time = time.time() - start_gen

    # Step 3: Compute total time
    total_time = classification_time + decomposition_time + retrieval_time + generation_time

    return {
        "classification": classification,
        "retrieved_context": context,
        "final_answer": final_answer,
        "timings": {
            "classification_time": classification_time,
            "decomposition_time": decomposition_time,
            "retrieval_time": retrieval_time,
            "generation_time": generation_time,
            "total_time": total_time
        }
    }


In [13]:
import pandas as pd
from tqdm import tqdm
import time

# Load your source CSV
df = pd.read_csv("../../../Dataset_code_csvs/hotpotQA/hotpotQA_dataset_versions/5884paras_598queries/Urdu/598_QnAs_translated.csv")

# Output CSV path
output_path = "../../results/pipeline results/5884paras_598qna/modLQR_processed_results.csv"

# Initialize storage
results = []
batch_times = []
total_start = time.time()

# Loop over each row
for idx, row in tqdm(df.iterrows(), total=len(df)):
    query = row["translated_question"]
    answer = row["translated_answer"]
    question_type = row["level"]

    start_time = time.time()

    try:
        result = multihop_handling_modLQR(query, question_type)

        classification = result["classification"]
        retrieved_context = result["retrieved_context"]
        final_answer = result["final_answer"]
        timings = result["timings"]
        total_time_one = timings["total_time"]

    except Exception as e:
        print(f"Error processing query {idx}: {e}")
        classification = "Error"
        retrieved_context = "Error"
        final_answer = "Error"
        timings = {
            "classification_time": 0,
            "decomposition_time": 0,
            "retrieval_time": 0,
            "generation_time": 0,
            "total_time": 0
        }
        total_time_one = 0

    elapsed = time.time() - start_time
    batch_times.append(elapsed)

    results.append({
        "translated_question": query,
        "translated_answer": answer,
        "classification": classification,
        "retrieved_context": retrieved_context,
        "final_answer": final_answer,
        "classification_time": timings["classification_time"],
        "decomposition_time": timings["decomposition_time"],
        "retrieval_time": timings["retrieval_time"],
        "generation_time": timings["generation_time"],
        "total_time": timings["total_time"],
        "level": question_type
    })

    print(f"Processed record {idx+1}/{len(df)} in {elapsed:.2f} seconds.")

    # Write batch every 100 queries
    if (idx + 1) % 100 == 0:
        pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(idx), index=False, encoding="utf-8-sig")
        avg_batch_time = sum(batch_times) / len(batch_times)
        print(f"\n--- Saved batch up to record {idx+1}")
        print(f"Average time for last 100 records: {avg_batch_time:.2f} seconds\n")
        results = []
        batch_times = []

# Save any remaining records
if results:
    pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(len(df) % 100), index=False, encoding="utf-8-sig")

# Final stats
total_elapsed = time.time() - total_start
avg_total_time = total_elapsed / len(df)
print(f"\n✅ All records processed.")
print(f"Total processing time: {total_elapsed:.2f} seconds.")
print(f"Average time per record: {avg_total_time:.2f} seconds.")


  0%|          | 1/598 [00:16<2:40:03, 16.09s/it]

Processed record 1/598 in 16.08 seconds.


  0%|          | 2/598 [00:25<2:02:08, 12.30s/it]

Processed record 2/598 in 9.64 seconds.


  1%|          | 3/598 [00:33<1:39:39, 10.05s/it]

...

Processed record 596/598 in 28.74 seconds.


100%|█████████▉| 597/598 [4:02:56<00:30, 30.46s/it]

Processed record 597/598 in 34.64 seconds.


100%|██████████| 598/598 [4:03:24<00:00, 24.42s/it]

Processed record 598/598 in 28.38 seconds.

✅ All records processed.
Total processing time: 14604.97 seconds.
Average time per record: 24.42 seconds.


In [ ]:
import pandas as pd
from tqdm import tqdm
import time

# Load your source CSV
df = pd.read_csv("../../../Dataset_code_csvs/hotpotQA/hotpotQA_dataset_versions/5884paras_598queries/Urdu/598_QnAs_translated.csv")

# Output CSV path
output_path = "../../results/pipeline results/5884paras_598qna/modLQR_processed_results.csv"

# Initialize storage
results = []
batch_times = []
total_start = time.time()

# Loop over each row
for idx, row in tqdm(df.iterrows(), total=len(df)):
    if idx < 98:
        continue  # Skip the first 98 records

    query = row["translated_question"]
    answer = row["translated_answer"]
    question_type = row["level"]

    start_time = time.time()

    try:
        result = multihop_handling_modLQR(query, question_type)

        classification = result["classification"]
        retrieved_context = result["retrieved_context"]
        final_answer = result["final_answer"]
        timings = result["timings"]
        total_time_one = timings["total_time"]

    except Exception as e:
        print(f"Error processing query {idx}: {e}")
        classification = "Error"
        retrieved_context = "Error"
        final_answer = "Error"
        timings = {
            "classification_time": 0,
            "decomposition_time": 0,
            "retrieval_time": 0,
            "generation_time": 0,
            "total_time": 0
        }

    elapsed = time.time() - start_time
    batch_times.append(elapsed)

    results.append({
        "translated_question": query,
        "translated_answer": answer,
        "classification": classification,
        "retrieved_context": retrieved_context,
        "final_answer": final_answer,
        "classification_time": timings["classification_time"],
        "decomposition_time": timings["decomposition_time"],
        "retrieval_time": timings["retrieval_time"],
        "generation_time": timings["generation_time"],
        "total_time": timings["total_time"],
        "level": question_type
    })

    print(f"Processed record {idx+1}/{len(df)} in {elapsed:.2f} seconds.")

    # Write batch every 100 queries
    if (idx + 1) % 100 == 0:
        pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(idx), index=False, encoding="utf-8-sig")
        avg_batch_time = sum(batch_times) / len(batch_times)
        print(f"\n--- Saved batch up to record {idx+1}")
        print(f"Average time for last 100 records: {avg_batch_time:.2f} seconds\n")
        results = []
        batch_times = []

# Save any remaining records
if results:
    pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(len(df) % 100), index=False, encoding="utf-8-sig")

# Final stats
total_elapsed = time.time() - total_start
avg_total_time = total_elapsed / (len(df) - 98)  # Adjusted for skipped records
print(f"\n✅ All applicable records processed.")
print(f"Total processing time: {total_elapsed:.2f} seconds.")
print(f"Average time per record: {avg_total_time:.2f} seconds.")


 17%|█▋        | 99/598 [02:14<11:18,  1.36s/it]

Processed record 99/598 in 134.66 seconds.


 17%|█▋        | 100/598 [04:34<27:38,  3.33s/it]

Processed record 100/598 in 139.77 seconds.

--- Saved batch up to record 100
Average time for last 100 records: 137.21 seconds

